In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import scipy.stats as st
from scipy.special import ndtri
from copy import deepcopy
from sklearn.metrics import cohen_kappa_score
from matplotlib.lines import Line2D

import sys
sys.path.insert(0, '/Users/Symptom_Documentation/')

from functions import helpers as hf 
from functions import helpers_plot as hp

In [ ]:
# load data

projectfolder = '/Users/Symptoms_Documentation/'
datafolder = projectfolder + 'data/'
figfolder = projectfolder + 'figures/'

df_survey = pd.read_csv(datafolder + 'by_person.csv')  # survey responses
df_demog = pd.read_csv(datafolder + 'data.csv')  # demographic info

In [ ]:
#### get race

info_filt = ~pd.isna(df_demog['mrn'])

hispanic_filt = df_demog['hispanic'] == 1
study_ids = df_demog.loc[info_filt, 'studyid']
study_id_hisp = df_demog.loc[info_filt & hispanic_filt, 'studyid'].to_list()
study_id_other = df_demog.loc[info_filt & ~hispanic_filt, 'studyid'].to_list()

print('Hispanic:', len(study_id_hisp), ', Non-Hispanic:', len(study_id_other))

## gender
# 0, Female | 1, Male | 2, Not reported

info_filt = ~pd.isna(df_demog['mrn'])
male_filt = df_demog['gender'] == 1
female_filt = df_demog['gender'] == 0
study_id_male = df_demog.loc[info_filt & male_filt, 'studyid'].to_list()
study_id_female = df_demog.loc[info_filt & female_filt, 'studyid'].to_list()

print('Male:', len(study_id_male), ', Female:', len(study_id_female))

# clean survey to contain only relevant columns

symptom_list = ['cramp', 'fatigue', 'musclesore', 'dryskin', 'itching']
pt_list = symptom_list
doc_list = [s + '_doc' for s in symptom_list]
nurse_list = [s + '_nurse' for s in symptom_list]
severity_list = [s + 'severity' for s in symptom_list]
df_survey_clean = df_survey[['studyid', 'date', 'date_nurse'] + pt_list + nurse_list + doc_list + severity_list]

# convert severity rating to integer from 0 to 4
df_survey_clean[severity_list] = np.round(df_survey_clean[severity_list]/25)

print('# rows:', len(df_survey_clean))
# display(df_survey_clean[0:50])

In [ ]:
# condense table so that 1 survey pair = 1 row

symptom_list = ['cramp', 'fatigue', 'musclesore', 'dryskin', 'itching']
pt_list = symptom_list
doc_list = [s + '_doc' for s in symptom_list]
nurse_list = [s + '_nurse' for s in symptom_list]
severity_list = [s + 'severity' for s in symptom_list]

df_final = pd.DataFrame()

for study_id in df_survey_clean['studyid'].unique():
    id_filt = df_survey_clean['studyid'] == study_id
    df = df_survey_clean.loc[id_filt, :]

    pt_filt = df['date'].notna()
    nurse_filt = df['date_nurse'].notna()
    doc_filt = df['date'].isna() & df['date_nurse'].isna()

    df_pt = df.loc[pt_filt, ['studyid', 'date'] + pt_list + severity_list].reset_index(drop=True)
    df_nurse = df.loc[nurse_filt, nurse_list].reset_index(drop=True)
    df_doc = df.loc[doc_filt, doc_list].reset_index(drop=True)
    df_doc['date'] = df_pt['date'].iloc[-1]

    df_combined = pd.concat([df_pt, df_nurse], axis=1)
    df_combined = df_combined.merge(df_doc, on='date', how='left')
    df_combined.insert(1, 'week', [i//3 + 1 for i in range(0, len(df_combined))])

    df_final = pd.concat([df_final, df_combined], ignore_index=True)

display(df_final[0:12])

In [ ]:
# condense table so that 1 row = 1 week

symptom_list = ['cramp', 'fatigue', 'musclesore', 'dryskin', 'itching']
pt_list = symptom_list
doc_list = [s + '_doc' for s in symptom_list]
nurse_list = [s + '_nurse' for s in symptom_list]
severity_list = [s + 'severity' for s in symptom_list]

df_byweek = pd.DataFrame()

for study_id in df_final['studyid'].unique():
    id_filt = df_final['studyid'] == study_id
    for week in [1,2,3,4]:
        week_filt = df_final['week'] == week
        df_pt_week = df_final.loc[id_filt & week_filt, :]
        
        pt_sym = df_pt_week.loc[:, pt_list].max()
        nurse_sym = df_pt_week.loc[:, nurse_list].max()
        doc_sym = df_pt_week.loc[:, doc_list].max()
        sym_sev = df_pt_week.loc[:, severity_list].max()

        combined = pd.concat([pt_sym, nurse_sym, doc_sym, sym_sev])

        df = pd.DataFrame([combined.tolist()], columns=combined.index)
        df.insert(0, 'week', week)
        df.insert(0, 'study_id', study_id)

        df_byweek = pd.concat([df_byweek, df], ignore_index=True)

    # df_final = pd.concat([df_final, df_combined], ignore_index=True)

display(df_byweek[0:12])

In [ ]:
# Calculate Kappa scores between patient + nurse and patient + physician

symptom_list = ['cramp', 'fatigue', 'musclesore', 'dryskin', 'itching']
doc_list = [s + '_doc' for s in symptom_list]
nurse_list = [s + '_nurse' for s in symptom_list]

kappa_nurse = {}
kappa_doc = {}

for sym in symptom_list:
    print(sym)
    nurse_kappa_week = {}
    doc_kappa_week = {}
    for week in range(1,5):
        print(week)
        df_byweek_subset = df_byweek[df_byweek["week"] == week]
    
        pt_scores = df_byweek_subset[sym]
        print("Number of patient scores as NaN:", sum(np.isnan(pt_scores)))
        nurse_scores = df_byweek_subset[sym + "_nurse"]
        print("Number of nurse scores as NaN:", sum(np.isnan(nurse_scores)))
        nurse_kappa_week[week] = round(cohen_kappa_score(pt_scores, nurse_scores), 2)

        if week == 4:
            doc_scores = df_byweek_subset[sym + "_doc"]
            print("Number of doctor scores as Nan", sum(np.isnan(doc_scores)))
            doc_kappa_week[week] = round(cohen_kappa_score(pt_scores, doc_scores), 2)
        else:
            doc_kappa_week[week] = None
    kappa_nurse[sym] = nurse_kappa_week
    kappa_doc[sym] = doc_kappa_week

print(kappa_nurse)
print(kappa_doc)

kappa_nurse_df = pd.DataFrame(kappa_nurse)
kappa_doc_df = pd.DataFrame(kappa_doc)

display(kappa_nurse_df)
display(kappa_doc_df)

In [ ]:
def get_kappa_bysurvey(symp_pt, symp_clin, all_symptoms=['cramp', 'fatigue', 'musclesore', 'dryskin', 'itching']):
    
    # total number of symptoms that both raters said were present
    A = len(np.intersect1d(symp_pt, symp_clin))
    
    # total number of symptoms rater 2 said was absent but rater 1 said was present
    B = len(np.intersect1d(np.setdiff1d(all_symptoms, symp_clin), symp_pt))
    
    # total number of symptoms rater 1 said was absent but rater 2 said was present
    C = len(np.intersect1d(np.setdiff1d(all_symptoms, symp_pt), symp_clin))
    
    # total number of symptoms both raters said were absent
    D = len(np.intersect1d(np.setdiff1d(all_symptoms, symp_pt), np.setdiff1d(all_symptoms, symp_clin)))

    # print(A, B, C, D)

    # probability of agreement
    p_a = (A+D) / (A+B+C+D)
    p_correct = ((A+B)/(A+B+C+D)) * ((A+C)/(A+B+C+D))
    p_incorrect = ((C+D)/(A+B+C+D)) * ((B+D)/(A+B+C+D))
    p_e = p_correct + p_incorrect

    try:
        kappa = (p_a - p_e) / (1 - p_e)
    except ZeroDivisionError as e:
        kappa = np.nan
        # print(p_a - p_e, 1 - p_e)
        # print('division by zero error')
        
    return kappa

In [ ]:
# kappa between patient + nurse, patient + doctor per week

symptom_list = ['cramp', 'fatigue', 'musclesore', 'dryskin', 'itching']
pt_list = symptom_list
doc_list = [s + '_doc' for s in symptom_list]
nurse_list = [s + '_nurse' for s in symptom_list]

kappa_nurse_list = []
kappa_doc_list = []

for row in df_byweek.itertuples():
    pt_symptoms = []
    nurse_symptoms = []
    doc_symptoms = []
    for sym in symptom_list:
        if getattr(row, sym) == 1:
            pt_symptoms.append(sym)
        if getattr(row, sym + '_nurse') == 1:
            nurse_symptoms.append(sym)
        if getattr(row, sym + '_doc') == 1:
            doc_symptoms.append(sym)

    if len(doc_symptoms) > 0:
        kappa_doc = get_kappa_bysurvey(pt_symptoms, doc_symptoms, symptom_list)
    else:
        kappa_doc = np.nan
    
    kappa_nurse = get_kappa_bysurvey(pt_symptoms, nurse_symptoms, symptom_list)
    kappa_nurse_list.append(kappa_nurse)
    kappa_doc_list.append(kappa_doc)
            
df_byweek['kappa_nurse'] = kappa_nurse_list
df_byweek['kappa_doc'] = kappa_doc_list

In [ ]:
# graph the distribution of kappa and the average kappa across all symptoms
kappa_list = []

plt.figure(figsize=(12,3))
for i in range(1, 5):
    week_filt = df_byweek['week'] == i
    nurse_kappas = df_byweek.loc[week_filt, 'kappa_nurse']
    doc_kappas = df_byweek.loc[week_filt, 'kappa_doc']
    plt.subplot(1,4,i)
    plt.hist(nurse_kappas, range=[-1,1], bins=20)
    plt.ylim([0, 60])
    plt.title('Week ' + str(i))

    kappa_list.append({'Mean (nurse)': np.nanmean(nurse_kappas), \
                       'Std (nurse)': np.std(nurse_kappas), \
                       'Mean (physician)': np.nanmean(doc_kappas), \
                       'Std (physician)': np.std(doc_kappas)})

plt.tight_layout()
plt.show()

df_kappa = pd.DataFrame(kappa_list)
display(df_kappa)

In [ ]:
df_kappa.to_csv(projectfolder + 'results/kappa_byweek.csv')

In [ ]:
# create dictionary of symptoms as reported by patient, doctor, nurse, and NLP

uniq_sids = df_survey_clean['studyid'].unique()
symptom_dict = {}   # key = reporter+symptom, value=list of studyids
severity_dict = {}  # key1 = symptom, key2=severity, value = list of studyids w/ that symptom and severity

# create dictionary structure
for col in pt_list:
    severity_dict[col] = {}
    for i in range(0, 5):
        severity_dict[col][i] = []

for sid in uniq_sids:
    sid_filt = df_survey_clean['studyid'] == sid
    for col in pt_list:
        if np.sum(df_survey_clean.loc[sid_filt, col]) > 0:  # patient has reported symptom
            severity_label = col.split('_')[0] + 'severity'  
            severity = np.max(df_survey_clean.loc[sid_filt, severity_label]) # get patient-reported severity
            if not pd.isna(severity):
                severity_dict[col][severity].append(sid)
            else:
                severity_dict[col][0].append(sid)
            
for sid in uniq_sids:
    sid_filt = df_survey_clean['studyid'] == sid
    for col in nurse_list + doc_list:
        if np.sum(df_survey_clean.loc[sid_filt, col]) > 0:
            if col not in symptom_dict:
                symptom_dict[col] = []
            symptom_dict[col].append(sid)
            
# NLP results copied and pasted from NLP code

symptom_dict['cramp_nlp'] = [1, 3, 9, 11, 16, 18, 19, 20, 34, 36, 38, 50, 51, 54, 56, 57, 60, 65, 69, 70, 73, 76, 81, 82, 84, 100]
symptom_dict['fatigue_nlp'] = [8, 19, 33, 39, 42, 49, 61, 62, 75, 78, 79, 81, 89, 93]
symptom_dict['itching_nlp'] = [2, 3, 6, 9, 10, 11, 16, 31, 37, 38, 47, 54, 60, 62, 66, 75, 77, 83, 97, 100]
symptom_dict['musclesore_nlp'] = [1, 42, 66, 73, 76, 86, 88, 94, 95]
symptom_dict['dryskin_nlp'] = []

symptom_label_map = {'cramp': 'Cramp', 'fatigue': 'Fatigue', 'musclesore': 'Muscle Soreness', \
                     'itching': 'Itching', 'dryskin': 'Dry Skin'}
rater_label_map = {'nurse': 'Nurse', 'doc': 'Physician', 'nlp': 'NLP'}

In [ ]:
# FIGURE 2 - Bar Plot of Number of Reported Patients by Patient, Nurse, Physician, Nurse + Physician, and NLP

plt.figure(figsize=[9, 4])

# set width of bar
barWidth = 0.14

# only above certain severity
sev_cutoff = 0
keys_all = list(symptom_dict.keys())

counts_byrater = {'actual': [], 'nlp': [], 'doc': [], 'nurse': [], 'doc+nurse': []}
rater_order = ['actual', 'nurse', 'doc', 'doc+nurse', 'nlp']
rater_label_map = {'actual': 'Patient Survey', 'nurse': 'Nurse Survey', \
                   'doc': 'Physician Survey', 'nlp': 'NLP of EHR', 'doc+nurse': 'Nurse+Physician Surveys'}
color_map = {'actual': '#292787', 'nurse': '#68686b', 'doc': '#a35149', 'nlp': '#d1c886', 'doc+nurse': '#AFE1AF'}
symptom_label_map = {'cramp': 'Cramp', 'fatigue': 'Fatigue', 'musclesore': 'Muscle Soreness', \
                     'itching': 'Itching', 'dryskin': 'Dry Skin'}
symptom_order = ['fatigue', 'cramp', 'dryskin', 'musclesore', 'itching']

for i, sym in enumerate(symptom_order):
    sym_keys = [k for k in keys_all if sym in k]
    species = tuple(['pt']+[k.replace(sym+'_', '') for k in sym_keys])
    
    actual_sids = [severity_dict[sym][s] for s in severity_dict[sym] if s>=sev_cutoff]
    actual_sids = [*set(list(itertools.chain(*actual_sids)))]
    
    counts_byrater['actual'].append(len(actual_sids))
    
    for k in sym_keys:
        rater = k.split('_')[1]
        sids_byrater = symptom_dict[k]
        counts_byrater[rater].append(len(sids_byrater))

    sids_bynurse = symptom_dict[sym + '_nurse']
    sids_bydoc = symptom_dict[sym + '_doc']
    sids_both = np.unique(sids_bynurse + sids_bydoc)
    counts_byrater['doc+nurse'].append(len(sids_both))

for i, rater in enumerate(rater_order):
    barheights = counts_byrater[rater]
    if i==0:
        barposition = np.arange(len(barheights))
    else:
        barposition = [x+barWidth for x in prevbar]
    prevbar = barposition
    plt.bar(barposition, barheights, width=barWidth, label=rater_label_map[rater], \
            color=color_map[rater], edgecolor = 'black')

# # Adding Xticks
plt.xlabel('Symptom', fontweight ='bold', fontsize = 15)
plt.ylabel('# Reported Patients', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(symptom_order))], \
           [symptom_label_map[sym] for sym in symptom_order])
 
# plt.box(False)
plt.gca().spines['top'].set_color('none')
plt.gca().spines['right'].set_color('none')
plt.legend()

plt.savefig(figfolder + 'fig2_updated_v2.png', dpi=200,  bbox_inches='tight')
plt.show()


In [ ]:
# Creates a dataframe that contains unique study IDs and symptom presence based on severity thresholds and rater source
# Contains demographic information
all_ids = df_survey_clean['studyid'].unique()
df_sym = pd.DataFrame({'Study ID': all_ids})

sev_cutoff = 0
raters = ['doc', 'nurse', 'nlp']
for sym in severity_dict.keys():
    actual_ids = [severity_dict[sym][s] for s in severity_dict[sym] if s>=sev_cutoff]
    actual_ids = [*set(list(itertools.chain(*actual_ids)))]
    df_sym[sym + '_actual'] = df_sym['Study ID'].isin(actual_ids).to_list()
    
    for r in raters:
        rater_ids = symptom_dict[sym + '_' + r]
        df_sym[sym + '_' + r] = df_sym['Study ID'].isin(rater_ids).to_list()

df_sym['gender'] = 'female'
df_sym.loc[df_sym['Study ID'].isin(study_id_male), 'gender'] = 'male'
df_sym['race'] = 'hispanic'
df_sym.loc[df_sym['Study ID'].isin(study_id_other), 'race'] = 'nonhispanic'

highsev_ids = []
sev_min = 3
for sym in severity_dict:
    for sev in range(sev_min, 5):
        highsev_ids.extend(severity_dict[sym][sev])
    
highsev_ids = np.unique(highsev_ids)
df_sym['severity'] = 'low'
df_sym.loc[df_sym['Study ID'].isin(highsev_ids), 'severity'] = 'high'

display(df_sym.head())

In [ ]:
# Retrieve unique IDs of patients with high severity or low severity symptoms
highsev_ids = []
sev_min = 3
for sym in severity_dict:
    for sev in range(sev_min, 5):
        highsev_ids.extend(severity_dict[sym][sev])
    
highsev_ids = np.unique(highsev_ids)
lowsev_ids = np.setdiff1d(uniq_sids, highsev_ids)

In [ ]:
# plot percentage of patients reporting symptom
symptoms = ['fatigue', 'cramp', 'dryskin', 'musclesore', 'itching']
counts = []
for sym in symptoms:
    sym_count = 0
    for sev in severity_dict[sym]:
        if sev > 0:
            sym_count += len(severity_dict[sym][sev])
    counts.append(100*sym_count / len(uniq_sids))
    
plt.figure(figsize=(3,2.8))
plt.bar([symptom_label_map[sym] for sym in symptoms], counts, width=0.8)
plt.ylabel('% of patients reporting symptom')
# plt.xlabel('Symptom')
plt.xticks(rotation=90)
plt.gca().spines['top'].set_color('none')
plt.gca().spines['right'].set_color('none')
plt.autoscale()
plt.savefig(figfolder + 'fig1.pdf', format='pdf', dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
print(symptom_dict.keys())
print(severity_dict.keys())

In [ ]:
# Calculate and plots sensitivity, specificity, PPV, and NPV for patients based on rater source and symptom
all_ids = df_survey_clean['studyid'].unique()  # all patients
actual_ids = hf.get_ids_ACTUAL(severity_dict)  # all patients with ANY symptom reported

fig1 = hp.plot_combined(symptom_dict, all_ids, actual_ids, figfolder+'fig3a.png')

all_ids = df_survey_clean['studyid'].unique()
actual_ids_low = hf.get_ids_ACTUAL(severity_dict, 1, 2)
actual_ids_hi = hf.get_ids_ACTUAL(severity_dict, 3, 4)
actual_id_dict = hf.get_id_dict_ACTUAL(severity_dict)

fig2 = hp.plot_bysymptom(symptom_dict, all_ids, actual_id_dict, figfolder + 'fig3b.png')


In [ ]:
# Calculate and plots sensitivity, specificity, PPV, and NPV for patients based on rater source and symptom
# Severity low vs high

all_ids = df_survey_clean['studyid'].unique()
actual_ids_low = hf.get_ids_ACTUAL(severity_dict, 1, 2)
actual_ids_hi = hf.get_ids_ACTUAL(severity_dict, 3, 4)

actual_id_dict = {'Severity < 3': actual_ids_low, 'Severity ≥ 3': actual_ids_hi}
color_dict = {'Severity < 3': '#0000a7', 'Severity ≥ 3': '#c1272d'}

hp.plot_bygroup(symptom_dict, all_ids, actual_id_dict, color_dict, figfolder + 'fig4a_updated.png')

actual_id_dict = hf.get_id_dict_ACTUAL(severity_dict)

cohort_dict = {'Severity<3': highsev_ids, 'Severity≥3': lowsev_ids}
color_dict = {'Severity<3': '#0000a7', 'Severity≥3': '#c1272d'}

hp.plot_bysymptom_bygroup(symptom_dict, actual_id_dict, cohort_dict, color_dict, \
                          figfolder+'fig_supp3.png')

In [ ]:
# Calculate and plots sensitivity, specificity, PPV, and NPV for patients based on rater source and symptom
# non-hispanic vs hispanic

all_ids = df_survey_clean['studyid'].unique()

actual_id_dict = {'Non-Hispanic': study_id_other, 'Hispanic': study_id_hisp}
color_dict = {'Non-Hispanic': '#0000a7', 'Hispanic': '#c1272d'}

hp.plot_bygroup(symptom_dict, all_ids, actual_id_dict, color_dict, figfolder + 'fig4b_updated.png')

actual_id_dict = hf.get_id_dict_ACTUAL(severity_dict)

cohort_dict = {'Non-Hispanic': study_id_other, 'Hispanic': study_id_hisp}
color_dict = {'Non-Hispanic': '#0000a7', 'Hispanic': '#c1272d'}

hp.plot_bysymptom_bygroup(symptom_dict, actual_id_dict, cohort_dict, color_dict, \
                          figfolder+'fig_supp2_corrected.png')

In [ ]:
# Calculate and plots sensitivity, specificity, PPV, and NPV for patients based on rater source and symptom
# male vs female

all_ids = df_survey_clean['studyid'].unique()

actual_id_dict = {'Male': study_id_male, 'Female': study_id_female}
color_dict = {'Male': '#0000a7', 'Female': '#c1272d'}

hp.plot_bygroup(symptom_dict, all_ids, actual_id_dict, color_dict, figfolder + 'fig4c.png')

actual_id_dict = hf.get_id_dict_ACTUAL(severity_dict)

cohort_dict = {'Male': study_id_male, 'Female': study_id_female}
color_dict = {'Male': '#0000a7', 'Female': '#c1272d'}

figfolder = projectfolder + 'figures/'
hp.plot_bysymptom_bygroup(symptom_dict, actual_id_dict, cohort_dict, color_dict, \
                          filename=figfolder+'fig_supp1.png')

In [ ]:
# low vs high frequency of reported symptoms
# high frequency = pt reported symptoms >= 50% of the time; else low

freq_dict = {'highfreq': {}, 'lowfreq': {}}
for studyid in df_final['studyid'].unique():
    id_filt = df_final['studyid'] == studyid
    for sym in symptom_list:
        sym_count = df_final.loc[id_filt, sym].sum()
        if (sym_count / np.sum(id_filt)) >= 0.5:
            if sym not in freq_dict['highfreq']:
                freq_dict['highfreq'][sym] = []
            freq_dict['highfreq'][sym].append(studyid)
        else:
            if sym not in freq_dict['lowfreq']:
                freq_dict['lowfreq'][sym] = []
            freq_dict['lowfreq'][sym].append(studyid)
# high frequency vs low frequency reporting

all_ids = df_survey_clean['studyid'].unique()

highfreq_ids = sum([freq_dict['highfreq'][sym] for sym in freq_dict['highfreq']], [])
lowfreq_ids = sum([freq_dict['lowfreq'][sym] for sym in freq_dict['lowfreq']], [])

actual_id_dict = {'High Frequency': highfreq_ids, 'Low Frequency': lowfreq_ids}
color_dict = {'High Frequency': '#0000a7', 'Low Frequency': '#c1272d'}

hp.plot_bygroup(symptom_dict, all_ids, actual_id_dict, color_dict, figfolder + 'fig3c_updated.png')

In [ ]:
# physician survey,  nurse survey, NLP performance metrics (overall)
# sensitivity, specificity, PPV, NPV

all_ids = df_survey_clean['studyid'].unique()
actual_id_dict = hf.get_id_dict_ACTUAL(severity_dict)

hp.plot_bysymptom(symptom_dict, all_ids, actual_id_dict, figfolder + 'fig3b.png')

for rater in ['nurse', 'doc', 'nlp']:
    print('\n',rater)
    for sym in actual_id_dict:
        print(sym)
        pos_ids = actual_id_dict[sym]
        pred_pos_ids = symptom_dict[sym + '_' + rater]

        conf_mat = hf.get_confusion_matrix(pos_ids, pred_pos_ids, all_ids)
        print(conf_mat)
        for label in conf_mat:
            print(label, conf_mat[label] / 97)

In [ ]:
# plot by symptom, grouped by raters and reported frequency
# similar to hp.plot_bysymptom_group
    
symptom_label_map = {'cramp': 'Cramp', 'fatigue': 'Fatigue', 'musclesore': 'Muscle Soreness', \
                 'itching': 'Itching', 'dryskin': 'Dry Skin'}
rater_label_map = {'nurse': 'Nurse', 'doc': 'Physician', 'nlp': 'NLP'}
symptom_order = ['fatigue', 'cramp', 'dryskin', 'musclesore', 'itching'][::-1]
cohort_label_map = {'highfreq': 'High Frequency', 'lowfreq': 'Low Frequency'}
color_dict = {'highfreq': '#0000a7', 'lowfreq': '#c1272d'}

plt.figure(figsize=[8, 6])
plt.subplots_adjust(wspace=0.2, hspace=0)
cols = 4
rows = 1
raters = ['nlp', 'doc', 'nurse']  # Set specific rater order here
markers = ['o', 's', 'v']
marker_dict = dict(zip(raters, markers))
    
for c, cohort in enumerate(freq_dict):
    for i,rater in enumerate(raters):
        id_dict = hf.get_id_dict_byrater(symptom_dict, rater)

        # get stats by symptom
        count = 0
        for j,sym in enumerate(symptom_order):
            actual_ids = freq_dict[cohort][sym]
            ids_byrater = id_dict[sym]
            conf_mat = hf.get_confusion_matrix(actual_ids, ids_byrater, df_final['studyid'].unique())
            
            sens = hf.sensitivity(conf_mat)
            spec = hf.specificity(conf_mat)
            ppv = hf.PPV(conf_mat)
            npv = hf.NPV(conf_mat)
            sens_ci, spec_ci, ppv_ci, npv_ci = hf.confidence_interval(conf_mat, alpha=0.95)

            y_offset = -c*0.08
            y = j+(i*0.2)-0.2 + y_offset

            marker = marker_dict[rater]

            plt.subplot(rows,cols,1)
            plt.plot(sens_ci, (y,y), color=color_dict[cohort])
            plt.plot(sens, y, marker, color='black')
            plt.title('Sensitivity')
            plt.xlim([-0.05, 1.05])
            plt.yticks([r for r in range(len(symptom_order))], \
                       [symptom_label_map[s] for s in symptom_order])

            plt.subplot(rows,cols,2)
            plt.plot(spec_ci, (y,y), color=color_dict[cohort])
            plt.plot(spec, y, marker, color='black')
            plt.xlim([-0.05, 1.05])
            plt.title('Specificity')
            plt.tick_params(left = False)
            plt.yticks([])

            plt.subplot(rows,cols,3)
            plt.plot(ppv_ci, (y,y), color=color_dict[cohort])
            plt.plot(ppv, y, marker, color='black')
            plt.xlim([-0.05, 1.05])
            plt.title('PPV')
            plt.tick_params(left = False)
            plt.yticks([])

            plt.subplot(rows,cols,4)
            if j==0:
                plt.plot(npv_ci, (y,y), color=color_dict[cohort], label=cohort_label_map[cohort])
                plt.plot(npv, y, marker, color='black', label=rater_label_map[rater])
            else:
                plt.plot(npv_ci, (y,y), color=color_dict[cohort])
                plt.plot(npv, y, marker, color='black')
            plt.xlim([-0.05, 1.05])
            plt.title('NPV')
            plt.tick_params(left = False)
            plt.yticks([])

custom_legend_handles = [
    Line2D([0], [0], color=color_dict[cohort], marker=marker_dict[rater], markersize=6, 
            markerfacecolor='black', markeredgecolor='black', linestyle='-', 
            label=f"{rater_label_map[rater]} {cohort_label_map[cohort]}")
    for cohort in reversed(cohort_dict)
    for rater in raters 
]

custom_legend_handles = custom_legend_handles[::-1]

# Set the legend with custom handles
plt.legend(handles=custom_legend_handles, loc="upper right", bbox_to_anchor=(2.8, 1.0))
plt.savefig(figfolder+'fig_supp4_symfreq_updated.png', dpi=200,  bbox_inches='tight')
plt.show()


In [ ]:
# kappa between patient + nurse, patient + doctor, patient + NLP, and patient + nurse and physician

rater_label_map = {'nurse': 'Nurse', 'doc': 'Physician', 'nlp': 'NLP', 'nurse+doc': 'Nurse + Physician'}

all_ids = df_survey_clean['studyid'].unique()
actual_id_dict = hf.get_id_dict_ACTUAL(severity_dict)
# print(actual_id_dict)

raters = ['nurse', 'doc', 'nurse+doc', 'nlp']
symptom_order = ['fatigue', 'cramp', 'dryskin', 'musclesore', 'itching']
kappa_list = []
for i,rater in enumerate(raters):
    if rater == 'nurse+doc':
        id_dict_nurse = hf.get_id_dict_byrater(symptom_dict, 'nurse')
        id_dict_doc = hf.get_id_dict_byrater(symptom_dict, 'doc')
        id_dict = {}
        for sym in id_dict_nurse:
            id_dict[sym] = np.unique(id_dict_nurse[sym], id_dict_doc[sym])
    else:
        id_dict = hf.get_id_dict_byrater(symptom_dict, rater)
        
    kappa_dict = {}
    for sym in symptom_order:
        actual_ids = actual_id_dict[sym]
        rater_ids = id_dict[sym]
        kappa = hf.get_kappa(actual_ids, rater_ids, all_ids)
        kappa_dict[symptom_label_map[sym]] = kappa
    kappa_list.append(kappa_dict)
    
df_kappa = pd.DataFrame(kappa_list, index=[rater_label_map[r] for r in raters])
display(np.round(df_kappa, 3))

In [ ]:
df_kappa.round(2).to_csv(projectfolder + 'results/kappa_table.csv')